# 1. Preparations

* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.


In [11]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models


/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 527 (delta 170), reused 155 (delta 154), pack-reused 314 (from 1)
Receiving objects: 100% (527/527), 5.38 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (281/281), done.
/content/GFPGAN
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWar

In [16]:
import os
import cv2
import shutil
from google.colab import files
import fileinput
import glob
import matplotlib.pyplot as plt


In [ ]:
basicsr_path = "/usr/local/lib/python3.10/dist-packages/basicsr"
degradations_file = os.path.join(basicsr_path, "data", "degradations.py")

with fileinput.FileInput(degradations_file, inplace=True) as file:
    for line in file:
        if "from torchvision.transforms.functional_tensor import rgb_to_grayscale" in line:
            print("from torchvision.transforms.functional import rgb_to_grayscale", end="\n")
        else:
            print(line, end="")

In [17]:
upload_folder = 'inputs/upload'
frames_folder = 'inputs/frames'
results_folder = 'results/frames'
output_video_path = 'results/output_video.mp4'

In [18]:
os.makedirs(upload_folder, exist_ok=True)
os.makedirs(frames_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

In [19]:
uploaded = files.upload()
video_path = None
for filename in uploaded.keys():
    video_path = os.path.join(upload_folder, filename)
    print(f'Uploaded {filename} to {video_path}')
    shutil.move(filename, video_path)


Saving viddee.mp4 to viddee.mp4
Uploaded viddee.mp4 to inputs/upload/viddee.mp4


In [20]:
def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f'frame_{frame_count:05d}.png')
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    cap.release()
    print(f"Extracted {frame_count} frames from {video_path}")

extract_frames(video_path, frames_folder)

Extracted 284 frames from inputs/upload/viddee.mp4


In [21]:
!rm -rf results/frames
!python inference_gfpgan.py -i inputs/frames -o results/frames -v 1.3 -s 2 --bg_upsampler realesrgan



/usr/local/lib/python3.10/dist-packages/realesrgan/utils.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_path, map_location=torch.device('cpu

In [24]:
def frames_to_video(input_folder, output_path, fps=30):
    frame_files = sorted(glob.glob(os.path.join(input_folder, '*.png')))
    if not frame_files:
        print("No frames found to assemble into video.")
        return
    # Get frame size from the first frame
    frame = cv2.imread(frame_files[0])
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
    video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        video.write(frame)
    video.release()
    print(f"Video saved to {output_path}")

frames_to_video('results/frames/restored_imgs', output_video_path)

# Step 6: Download the final video
files.download(output_video_path)


Video saved to results/output_video.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input Frame', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('GFPGAN Processed Frame', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Display a few sample frames
input_frames = sorted(glob.glob(os.path.join(frames_folder, '*.png')))
output_frames = sorted(glob.glob(os.path.join(results_folder, '*.png')))

for input_path, output_path in zip(input_frames[:5], output_frames[:5]):
    img_input = imread(input_path)
    img_output = imread(output_path)
    display(img_input, img_output)